# **Spatiotemporal models**

**Contains:** Spatiotemporal architecture

## Setup

In [59]:
# Import modules

# STUDY
# Utilities
import sys
sys.path.append("../transformers/")
import warnings
warnings.simplefilter("ignore")
import os, cv2

# Data
import numpy as np
import pandas as pd
from PIL import Image

# Visualization
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300

# Model
from fftstack import FFTTransformer

# Evaluation
from sklearn.metrics import r2_score, mean_squared_error
from scipy import stats

In [66]:
# Define convenience functions
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

def nrmse(true, pred):
    return mean_squared_error(true, pred, squared=False)/float(true.mean())

def yearly_linreg_p(true, pred):
    df = pd.concat((true, pred), axis=1).groupby(true.index.year)
    rmse_ = []
    for row in df:
        rmse_.append(rmse(row[1].iloc[:, 0], row[1].iloc[:, 1]))
    yearly_true_rmse = pd.Series(rmse_, index=list(true.index.year.drop_duplicates()))
    slope, intercept, r_value, p_value, std_err = stats.linregress(yearly_true_rmse.values, yearly_true_rmse.index)
    return p_value

def in_range(start, end, step=1):
    return list(range(start, end+step, step))

def make_dir(root, name):
    return root+name

def build_directory(directory):
    directory_array = list(filter(None, (f"/{directory}").split("/")[1:]))
    for i in range(1, len(directory_array) + 1):
        try:
            os.mkdir("/".join(directory_array[0:i]))
        except FileExistsError:
            pass

def make_video(img_dir, save_dir):
    video_name = save_dir + '.mp4'
    images = [img for img in os.listdir(img_dir) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(img_dir, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name, 0, 5, (width,height), 0)
    for image in images:
        video.write(cv2.imread(os.path.join(img_dir, image)))
    cv2.destroyAllWindows()
    video.release()

## Data

In [26]:
fetch_folder_dir = "data/sie-csv/"

pixelwise_data = []
for file in os.listdir(fetch_folder_dir):
    year, month = file[2:6], file[6:8]
    cur_img = list(np.loadtxt(fetch_folder_dir + file, delimiter=",", dtype=int).reshape(1,136192)[0])
    pixelwise_data.append([f"{year}-{month}"] + cur_img)
    print(file[0:len(file)-4], end="\r")
    
pixelwise_data = pd.DataFrame(pixelwise_data, columns=["date"]+list(range(136192))).set_index("date", drop=True)
pixelwise_data.index = pd.to_datetime(pixelwise_data.index, infer_datetime_format=True)
pixelwise_data = pixelwise_data.asfreq("MS").fillna(0)

col_names = pixelwise_data.columns
train, test = pixelwise_data.iloc[:408,], pixelwise_data.iloc[408:512,]

print(
    "\ntrain: ", len(train),
    "\ntest: ", len(test)
)

train:  408 ent_v3.0
test:  104


## Model

In [27]:
# FFT pipeline
fft_pipe = FFTTransformer(train_size=0.8,
                          iter_grid=np.arange(1,3),
                          threshold_grid=np.arange(0.1, 1.01, 0.2),
                          method="normalize",
                          performance_threshold=0.1,
                          verbose=True)

In [30]:
fft_train = fft_pipe.fit_transform(train)
fft_test = fft_pipe.transform(test)

build_directory("results/de-cycled/")
fft_train.to_csv("results/de-cycled/fft_train.csv")
fft_test.to_csv("results/de-cycled/fft_test.csv")

Scanning search space...
Training best models...


## Predictions

In [56]:
inversed_train = fft_pipe.inverse_transform(fft_train).round()
inversed_test = fft_pipe.inverse_transform(fft_test).round()

build_directory("results/final-predictions/")
inversed_train.to_csv("results/final-predictions/fft-train.csv")
inversed_test.to_csv("results/final-predictions/fft-test.csv")

## Evaluation

In [49]:
class EvaluationTable:
    def __init__(self, metrics, index_col="model"):
        self.metric_names = metrics.keys()
        self.metric_funcs = metrics.values()
        self.table = pd.DataFrame(columns=self.metric_names)

    def addEvaluation(self, name, true, pred):
        metric_results = []
        for metric in self.metric_funcs:
            metric_results.append(metric(true, pred))
        self.table.loc[name] = metric_results

    def loadTable(self):
        return self.table

In [36]:
fft_train = pd.read_csv("results/de-cycled/fft-train.csv").set_index("date", drop=True)
fft_test = pd.read_csv("results/de-cycled/fft-test.csv").set_index("date", drop=True)
fft_train.index = pd.to_datetime(fft_train.index, infer_datetime_format=True)
fft_train.columns = list(range(136192))
fft_test.index = pd.to_datetime(fft_test.index, infer_datetime_format=True)
fft_test.columns = list(range(136192))

In [41]:
def calc_ice_extent(array, areas):
    array = array / 100.00
    tot_ice_extent = np.sum(np.multiply(array, areas), axis=(1,2))
    return tot_ice_extent

area_size = np.fromfile("data/psn25area_v3.dat", dtype=np.dtype('<i')).reshape(448,304)/1000.0

fft_train_SIE = calc_ice_extent(np.array(fft_train, dtype=np.uint8).reshape(408, 448, 304), area_size)
fft_test_SIE = calc_ice_extent(np.array(fft_test, dtype=np.uint8).reshape(104, 448, 304), area_size)

In [57]:
metrics = {"rmse": rmse,
           "yearly_linreg_p": yearly_linreg_p}

fft_stack_eval = EvaluationTable(metrics)

fft_stack_eval.addEvaluation("fft-train-rmse", pd.Series(np.zeros(408), index=train.index), pd.Series(fft_train_SIE, index=train.index))
fft_stack_eval.addEvaluation("fft-test-rmse", pd.Series(np.zeros(104), index=test.index), pd.Series(fft_test_SIE, index=test.index))

results_table = fft_stack_eval.loadTable()
results_table

,rmse,yearly_linreg_p
fft-train-rmse,350914.449859,0.414254
fft-test-rmse,359658.591198,0.098330


## Visualization

In [64]:
build_directory("results/final-predictions/decycled-train-img/")
for num, row in enumerate(fft_train.index.date):
    post_img = (np.array(fft_train.iloc[num], dtype=np.uint8)*255).reshape(448, 304)
    Image.fromarray(post_img, mode="L").save(f"results/final-predictions/decycled-train-img/{str(row)}.png")

build_directory("results/final-predictions/decycled-test-img/")
for num, row in enumerate(fft_test.index.date):
    post_img = (np.array(fft_test.iloc[num], dtype=np.uint8)*255).reshape(448, 304)
    Image.fromarray(post_img, mode="L").save(f"results/final-predictions/decycled-test-img/{str(row)}.png")

In [65]:
build_directory("results/final-predictions/final-train-img/")
for num, row in enumerate(inversed_train.index.date):
    post_img = (np.array(inversed_train.iloc[num], dtype=np.uint8)*255).reshape(448, 304)
    Image.fromarray(post_img, mode="L").save(f"results/final-predictions/final-train-img/{str(row)}.png")

build_directory("results/final-predictions/final-test-img/")
for num, row in enumerate(inversed_test.index.date):
    post_img = (np.array(inversed_test.iloc[num], dtype=np.uint8)*255).reshape(448, 304)
    Image.fromarray(post_img, mode="L").save(f"results/final-predictions/final-test-img/{str(row)}.png")

In [67]:
make_video("results/final-predictions/decycled-train-img/", "results/final-predictions/decycled-train-vid")
make_video("results/final-predictions/decycled-test-img/", "results/final-predictions/decycled-test-vid")
make_video("results/final-predictions/final-train-img/", "results/final-predictions/final-train-vid")
make_video("results/final-predictions/final-test-img/", "results/final-predictions/final-test-vid")